# Dataset 20 Phase 1 ML

In [1]:
import pandas as pd 
import numpy as np
import time
import progressbar

# Feature extraction libraries 

import geopy
from geopy.geocoders import Nominatim

from IPy import IP
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from sklearn.model_selection import train_test_split
from sklearn import svm

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

pd.options.display.max_columns = 500

/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Importing Dataset 20

In [2]:
dataset_20_test = pd.read_csv('./datasets_for_project/dataset_20/\
Webpages_Classification_test_data.csv/Webpages_Classification_test_data.csv',  
                              engine='python')

dataset_20_train = pd.read_csv('./datasets_for_project/dataset_20/\
Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv',  
                               engine='python',
                               error_bad_lines=False)

dataset_20 = pd.concat([dataset_20_train, dataset_20_test], axis=0).drop(columns=['Unnamed: 0'])

Skipping line 951643: unexpected end of data


In [3]:
index_array = pd.Series([i for i in range(len(dataset_20))])
dataset_20 = dataset_20.set_index(index_array)

In [4]:
print("######### FEATURES #########")
print(dataset_20.columns)
print('\n')
print("######### Dataset details #########")
print('Contains raw data')
print(f"Total number of samples = {len(dataset_20)}")
print(f"Malicious Samples = {len(dataset_20[dataset_20['label']=='bad'])}")
print(f"Benign Samples = {len(dataset_20[dataset_20['label']=='good'])}")
dataset_20.head()

######### FEATURES #########
Index(['url', 'url_len', 'ip_add', 'geo_loc', 'tld', 'who_is', 'https',
       'js_len', 'js_obf_len', 'content', 'label'],
      dtype='object')


######### Dataset details #########
Contains raw data
Total number of samples = 1313575
Malicious Samples = 29778
Benign Samples = 1283797


,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good


## Converting locations to coordinates 

In [5]:
def get_lat_long(unique_locations):
    latitudes = []
    longitudes = []
    geolocator = Nominatim(user_agent="location_conversion")
    with progressbar.ProgressBar(max_value=len(unique_locations)) as bar:
        for index, location in enumerate(unique_locations):
            location_info = geolocator.geocode(location)
            latitudes.append(location_info.latitude)
            longitudes.append(location_info.longitude)
            bar.update(index)
    return unique_locations, latitudes, longitudes

unique_locations, latitudes, longitudes = get_lat_long(dataset_20['geo_loc'].unique())

100% (233 of 233) |######################| Elapsed Time: 0:01:56 Time:  0:01:56


In [6]:
def latitude_long_to_df(unique_locations, latitudes, longitudes, dataset):
    dataset_20['latitude'] = pd.Series(None)
    dataset_20['longitude'] = pd.Series(None)
    with progressbar.ProgressBar(max_value=len(unique_locations)) as bar:
        for index, location in enumerate(unique_locations):
            dataset_locs = dataset[dataset['geo_loc'] == location]
            dataset_locs['latitude'] = latitudes[index]
            dataset_locs['longitude'] = longitudes[index]
            
            dataset['latitude'].iloc[dataset_locs.index] = dataset_locs['latitude']
            dataset['longitude'].iloc[dataset_locs.index] = dataset_locs['longitude']
        return dataset

dataset_20 = latitude_long_to_df(unique_locations, latitudes, longitudes, dataset_20)

/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.p

In [7]:
dataset_20 = dataset_20.drop(columns=['geo_loc'])
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882


## Checking for existence of IP in URLs

In [8]:
dataset_20['has_IP_in_URL'] = pd.Series([0 for i in range(len(dataset_20))])

In [9]:
def check_if_URLS_have_ips(dataset):
    with progressbar.ProgressBar(max_value=len(dataset)) as bar:
        for i in dataset.index:
            split_string = re.split('/|.com', dataset['url'].iloc[i])
            for word in split_string:
                try:
                    IP(word)
                    dataset.iloc[i]['has_IP_in_URL'] = 1
                except:
                    pass
            bar.update(i)
    return dataset

In [10]:
dataset_20_tmp = check_if_URLS_have_ips(dataset_20)

N/A% (0 of 1313575) |                    | Elapsed Time: 0:00:00 ETA:  --:--:--/Users/ahmadchaiban/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100% (1313575 of 1313575) |##############| Elapsed Time: 0:01:06 Time:  0:01:06


In [11]:
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0


In [12]:
dataset_20['has_IP_in_URL'].unique()

array([0])

**None of these datasets have IP addresses in the URLs, might be a good idea to add some at some point**

## Number of subdomains

In [41]:
dataset_20['number_subdomains'] = pd.Series([0 for i in dataset_20.index])

In [42]:
def get_number_subdomains(url_string):
    removed_http = url_string.replace('http://', '').replace('https://', '')
    sub_array = removed_http.split('/')[0].replace('.com','').split('.')
    return len(sub_array) - 1

In [43]:
dataset_20['number_subdomains'] = dataset_20['url'].progress_apply(lambda x: get_number_subdomains(x))

In [47]:
dataset_20.head()

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0,1
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0,1
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0,2
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0,1


In [45]:
dataset_20['number_subdomains'].unique()

array([1, 2, 0, 3, 4, 5, 6, 7])

**Seems to be a good range of subdomains** 

## Hostname

In [82]:
def get_hostname(url):
    parsed_url = urllib.parse.urlparse(url)
    return parsed_url.netloc

dataset_20['hostname'] = dataset_20['url'].progress_apply(lambda x: get_hostname(x))

## Length of Hostname

In [121]:
def get_length_of_hostname(url):
    parsed_url = urllib.parse.urlparse(url)
    return len(parsed_url.netloc)

In [122]:
dataset_20['length_hostname'] = dataset_20['url'].progress_apply(lambda x: get_length_of_hostname(x))

In [130]:
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225,0,members.tripod.com,0.5


## Ratio of digits in URL

In [131]:
def get_ratio_digits_url(url):
    try: 
        return sum(c.isdigit() for c in dataset_20['url'].iloc[1])/len(url)
    except:
        return 0

dataset_20['ratio_digits_url'] = dataset_20['url'].progress_apply(lambda x: get_ratio_digits_url(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225,0,members.tripod.com,0.5


## Having @ Symbol

In [129]:
def check_at_symbol(url):
    if '@' in url:
        return 1
    return 0

dataset_20['having_@_in_url'] = dataset_20['url'].apply(lambda x: check_at_symbol(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225,0,members.tripod.com,0.5


In [126]:
dataset_20['having_@_in_url'].value_counts()

0    1313214
1        361
Name: having_@_in_url, dtype: int64

## Ratio of digits in Hostname

In [132]:
dataset_20['ratio_digits_hostname'] = dataset_20['hostname'].progress_apply(lambda x: get_ratio_digits_url(x))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225,0,members.tripod.com,0.5


## Number of underscores

In [135]:
dataset_20['number_underscores'] = dataset_20['url'].progress_apply(lambda x: x.count('_'))
dataset_20.head(1)

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225,0,members.tripod.com,0.5,0


In [161]:
dataset_20.to_csv('dataset_20_new_features.csv')

## Charset 

## DNS Query times

In [167]:
# import dns.resolver
# import time

# def dns_query_time_seconds(url):
#     print(url)
#     dns_start = time.perf_counter()
#     answers = dns.resolver.query(url)
#     dns_end = time.perf_counter()
#     return dns_end - dns_start

# dataset_20['dns_query_times_seconds'] = dataset_20['url'].progress_apply(lambda x: dns_query_time_seconds(x))
# dataset_20['dns_query_times_seconds'].head()

## Train-test-split

In [169]:
dataset_20

,url,url_len,ip_add,tld,who_is,https,js_len,js_obf_len,content,label,latitude,longitude,has_IP_in_URL,number_subdomains,length_hostname,ratio_digits_url,having_@_in_url,hostname,ratio_digits_hostname,number_underscores
0,http://members.tripod.com/russiastation/,40,42.77.221.155,com,complete,yes,58.0,0.000,Named themselves charged particles in a manly ...,good,23.973937,120.982018,0,1,18,0.225000,0,members.tripod.com,0.500000,0
1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,com,complete,yes,52.5,0.000,And filipino field \n \n \n \n \n \n \n \n the...,good,39.783730,-100.445882,0,1,11,0.281250,0,www.ddj.com,0.818182,0
2,http://www.naef-usa.com/,24,24.232.54.41,com,complete,yes,103.5,0.000,"Took in cognitivism, whose adherents argue for...",good,-34.996496,-64.967282,0,1,16,0.375000,0,www.naef-usa.com,0.562500,0
3,http://www.ff-b2b.de/,21,147.22.38.45,de,incomplete,no,720.0,532.800,fire cumshot sodomize footaction tortur failed...,bad,39.783730,-100.445882,0,2,13,0.428571,0,www.ff-b2b.de,0.692308,0
4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,com,complete,yes,46.5,0.000,"Levant, also monsignor georges. In 1800, lists...",good,39.783730,-100.445882,0,1,11,0.257143,0,us.imdb.com,0.818182,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313570,http://www.allnetporn.com/asians/t17_43/netver...,55,200.200.169.58,com,incomplete,no,407.7,289.467,roundeye poverty welfare lovejuice shitter pec...,bad,-10.333333,-53.200000,0,1,18,0.163636,0,www.allnetporn.com,0.500000,1
1313571,http://members.shaw.ca/brianholden,34,217.114.245.202,ca,complete,yes,163.5,0.000,"Be mixed southeast alaska.. Corals, and 1936 s...",good,64.686314,97.745306,0,2,15,0.264706,0,members.shaw.ca,0.600000,0
1313572,http://members.tripod.com/sjfloats/,35,191.49.186.181,com,incomplete,yes,32.0,0.000,"For 32 guadix, spain, as well as china and ind...",good,-10.333333,-53.200000,0,1,18,0.257143,0,members.tripod.com,0.500000,0
1313573,http://www.holyokemachine.com/,30,202.100.138.248,com,complete,yes,0.0,0.000,Common arrangement involves determining what p...,good,35.000074,104.999927,0,1,22,0.300000,0,www.holyokemachine.com,0.409091,0


In [170]:
X = dataset_20.drop(columns=['url', 'label'])
y = dataset_20['label']

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## SVM

In [159]:
svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = svm_model.predict(X_test)
accuracy_score(y_test, y_pred)

## Running selenium to get URL images

In [1]:
# chromeOptions = Options()
# chromeOptions.headless = True

# def get_screenshot(url, index):
#     driver = webdriver.Chrome()
#     driver.get(url)
#     time.sleep(1)

#     driver.get_screenshot_as_file('./dataset_20_screens/' + str(index) + '.png')
#     driver.quit()
    
# for i in dataset_20.index:
#     try:
#         get_screenshot(dataset_20.iloc[i]['url'], i)
#     except:
#         print(i)